In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import keras
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import layers
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json,load_model,Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.utils import layer_utils
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, UpSampling2D , Conv2D, MaxPool2D, MaxPooling2D,Input,Concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D,ZeroPadding2D,AveragePooling2D,Reshape,merge,Convolution2D
%matplotlib inline
from keras import layers
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
from tqdm import tqdm
import efficientnet.keras as efn 
from keras.layers import Dense
from keras.optimizers import Adam, Nadam
from keras_radam import RAdam
from keras import backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qa

In [2]:
grounds = 'Ground_Truths/'
originals = 'Originals/'
tcga_prediction_file = 'Kwok_A0CP_Predictions.csv'
tcga_patches_direction = '../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2C-654C2AA4C45F'
prediction_directory = 'Kwok_Predictions/'
probability_dir = 'Kwok_Probability_Map/'

In [4]:
def generate_results(grounds = 'Ground_Truths/', originals = 'Originals/', tcga_prediction_file = 'kwok_A0CP_Predictions.csv',
tcga_patches_direction = '../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2C-654C2AA4C45F', 
    prediction_directory = 'Kwok_Predictions/', probability_dir = 'Kwok_Probability_Map/', no_org = True, map_on_whole=True):
    svs_current = pd.read_csv(tcga_prediction_file)
    x = svs_current["filename"].str.split("_", n = 4, expand = True)
    svs_current["SVS_NO"] = x[x.columns[-4]]
    svs_current["ROW"] = x[x.columns[-3]]
    svs_current["COL"] = x[x.columns[-2]]
    svs_current["ROW"] = svs_current["ROW"].astype('int')
    svs_current["COL"] = svs_current["COL"].astype('int')
    x = svs_current["SVS_NO"].str.split("-", n = 5, expand = True)
    svs_current["SVS_NO"] = x[x.columns[2]]
    svs_current = svs_current.sort_values(['SVS_NO','ROW', 'COL'])
    total_rows = svs_current.ROW.max() #ROWS
    total_cols = svs_current.COL.max() #COLS
    classes = svs_current.Class_Name.values
    preds = svs_current.kwok_predictions_epochs_3.values
    SVS_NAME = svs_current.SVS_NO.unique()[0]
    
    #PREDICTIONS
    i = 0
    r = 0
    while r <= total_rows:
        c = 0
        while c <= total_cols:
            temp = np.zeros(shape=(5,5))
            temp = temp + preds[i]
            if c == 0:
                each_complete_row = temp.copy()
            else:
                each_complete_row = np.column_stack([each_complete_row, temp])
            c += 1
            i += 1
        if r == 0:
            cummulative_rows = each_complete_row.copy()
        else:
            cummulative_rows = np.row_stack([cummulative_rows, each_complete_row])
        r += 1
    w = min(cummulative_rows.shape[0], cummulative_rows.shape[1])
    cummulative_rows[cummulative_rows>=0.5] = 1
    cummulative_rows[cummulative_rows<0.5] = 0
    cummulative_rows = cv2.resize(cummulative_rows, (w, w), interpolation=cv2.INTER_AREA)
    blank_img = np.zeros(shape=(cummulative_rows.shape[0], cummulative_rows.shape[1], 3))
    blank_img[:, :, 0][cummulative_rows==0] = 255.0
    blank_img[:, :, 1][cummulative_rows==0] = 255.0
    blank_img[:, :, 2][cummulative_rows==0] = 255.0
    prediction_sample = cummulative_rows.copy()
    save_name = prediction_directory + '/' + SVS_NAME +'.png'
    cv2.imwrite(save_name, blank_img)
    
    #GROUND_TRUTH
    i = 0
    r = 0
    while r <= total_rows:
        c = 0
        while c <= total_cols:
            temp = np.zeros(shape=(5,5))
            if classes[i] != 0:
                temp = temp + 1.0
            if c == 0:
                each_complete_row = temp.copy()
            else:
                each_complete_row = np.column_stack([each_complete_row, temp])
            c += 1
            i += 1
        if r == 0:
            cummulative_rows = each_complete_row.copy()
        else:
            cummulative_rows = np.row_stack([cummulative_rows, each_complete_row])
        r += 1
    w = min(cummulative_rows.shape[0], cummulative_rows.shape[1])
    cummulative_rows = cv2.resize(cummulative_rows, (w, w), interpolation=cv2.INTER_AREA)
    blank_img = np.zeros(shape=(cummulative_rows.shape[0], cummulative_rows.shape[1], 3))
    blank_img[:, :, 0][cummulative_rows==0] = 255.0
    blank_img[:, :, 1][cummulative_rows==0] = 255.0
    blank_img[:, :, 2][cummulative_rows==0] = 255.0
    save_name = grounds + '/' + SVS_NAME +'.png'
    cv2.imwrite(save_name, blank_img)
    
    if no_org:
        #ORIGINAL
        class_mod = classes.copy()
        class_mod[class_mod == 1] = -1
        i = 0
        r = 0
        while r <= total_rows:
            c = 0
            while c <= total_cols:
                patch_name = tcga_patches_direction + '_' + str(r) + '_' + str(c) + '_' + str(class_mod[i]) + '.png'
                temp = cv2.imread(patch_name)
#                print (patch_name)
                temp = cv2.resize(temp, (5,5), interpolation=cv2.INTER_AREA)
                if c == 0:
                    each_complete_row = temp.copy()
                else:
                    each_complete_row = np.column_stack([each_complete_row, temp])
                c += 1
                i += 1
            if r == 0:
                cummulative_rows = each_complete_row.copy()
            else:
                cummulative_rows = np.row_stack([cummulative_rows, each_complete_row])
            r += 1
        w = min(cummulative_rows.shape[0], cummulative_rows.shape[1])
        cummulative_rows = cv2.resize(cummulative_rows, (w, w), interpolation=cv2.INTER_AREA)
        save_name = originals + '/' + SVS_NAME +'.png'
        cv2.imwrite(save_name, cummulative_rows)
    
    #PROBABILITY_MAP
    i = 0
    r = 0
    while r <= total_rows:
        c = 0
        while c <= total_cols:
            temp = np.zeros(shape=(5,5,3))
            if preds[i] < 0.5:
                temp[:, :, 0] = (1-preds[i]) * 255.0
                temp[:, :, 1] = preds[i] * 255.0
                temp[:, :, 2] = preds[i] * 255.0
            else:
                temp[:, :, 0] = preds[i] * 255.0
                temp[:, :, 1] = preds[i] * 255.0
                temp[:, :, 2] = (1-preds[i]) * 255.0
            if c == 0:
                each_complete_row = temp.copy()
            else:
                each_complete_row = np.column_stack([each_complete_row, temp])
            c += 1
            i += 1
        if r == 0:
            cummulative_rows = each_complete_row.copy()
        else:
            cummulative_rows = np.row_stack([cummulative_rows, each_complete_row])
        r += 1
    w = min(cummulative_rows.shape[0], cummulative_rows.shape[1])
    cummulative_rows = cv2.resize(cummulative_rows, (w, w), interpolation=cv2.INTER_AREA)
    if map_on_whole:
        cummulative_rows[:, :, 0][prediction_sample == 0] = 255.0
        cummulative_rows[:, :, 1][prediction_sample == 0] = 255.0
        cummulative_rows[:, :, 2][prediction_sample == 0] = 255.0
    save_name = probability_dir + '/' + SVS_NAME +'.png'
    cv2.imwrite(save_name, cummulative_rows)

In [5]:
generate_results(no_org=True, map_on_whole=True)

In [6]:
generate_results(tcga_prediction_file='kwok_A0BD_predictions.csv', tcga_patches_direction='../../Further_Patches/TCGA-BH-A0BD-01Z-00-DX1.CD4A6FC2-BA8C-4E30-972A-E6CD1BEAD8AD')

In [7]:
generate_results(tcga_prediction_file='kwok_A0CQ_predictions.csv', tcga_patches_direction='../../Further_Patches/TCGA-A2-A0CQ-01Z-00-DX1.4E5FB4E5-A08C-4C87-A3BE-0640A95AE649')

In [8]:
generate_results(tcga_prediction_file='kwok_A0BO_predictions.csv', tcga_patches_direction='../../Further_Patches/TCGA-BH-A0BO-01Z-00-DX1.1A704471-FEB3-40F9-9838-3E347A18285F')